In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17076421887197829738
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10100539392
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18083195606445087855
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.device('/device:GPU:0')

In [3]:
import tensorflow as tf
import numpy as np


class ResnetBlock(tf.keras.layers.Layer):

  def __init__(self, filters, strides = 1, activation = 'relu'):
    super().__init__()
    self.activation = tf.keras.activations.get(activation)
    self.main_layers = [
      tf.keras.layers.Conv2D(filters, kernel_size = 3, strides = strides, padding='same'),
      tf.keras.layers.BatchNormalization(),
      self.activation,
      tf.keras.layers.Conv2D(filters, kernel_size = 3, strides = 1, padding = 'same'),
      tf.keras.layers.BatchNormalization()
    ]
    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
          tf.keras.layers.Conv2D(filters, 1, strides = strides, padding = 'same'),
          tf.keras.layers.BatchNormalization()
      ]
    
  def call(self, inputs):
    input = inputs
    for layer in self.main_layers:
      input = layer(input)
    skip_input = inputs
    for layer in self.skip_layers:
  
      skip_input = layer(skip_input)
      
    return self.activation(input + skip_input)
    

In [4]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 7, input_shape=[224,224,3], strides=2, padding = "same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size = 3, strides=2, padding="same"))

before_filters=64
for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
  strides = 1 if filters==before_filters else 2
  model.add(ResnetBlock(filters, strides))
  before_filters = filters
model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation="softmax"))


In [5]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


In [6]:
x_train = tf.image.resize(x_train, [224,224])
x_test = tf.image.resize(x_test, [224,224])
x_train = x_train[:-10000]
y_train = y_train[:-10000]
x_val = x_train[-10000:]
y_val = y_train[-10000:]

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run StridedSlice: Dst tensor is not initialized. [Op:StridedSlice] name: strided_slice/

In [37]:
x_val.shape

(10000, 32, 32, 3, 1)

In [38]:


optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 112, 112, 64)      9472      
                                                                 
 batch_normalization_36 (Bat  (None, 112, 112, 64)     256       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 resnet_block_16 (ResnetBloc  (None, 56, 56, 64)       74368     
 k)                                                              
                                                      

c:\Users\sskym\Anaconda3\envs\tf_gpu\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [40]:
history = model.fit(x_train,y_train, epochs = 20, validation_data=(x_val, y_val))

Epoch 1/20
1563/1563 [==============================] - ETA: 0s - loss: 0.9817 - accuracy: 0.6534

ValueError: in user code:

    File "c:\Users\sskym\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\sskym\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sskym\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\sskym\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 1665, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\sskym\Anaconda3\envs\tf_gpu\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\sskym\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_i

In [14]:
X_test = tf.image.resize(x_test, [224,224])
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
79/79 [==============================] - 22s 259ms/step - loss: 0.8187 - accuracy: 0.8430
test loss, test acc: [0.8187060356140137, 0.8429999947547913]


In [8]:
!pip install tensorflow.keras.applications.resnet18

ERROR: Could not find a version that satisfies the requirement tensorflow.keras.applications.resnet18 (from versions: none)
ERROR: No matching distribution found for tensorflow.keras.applications.resnet18
